# Setup

In [ ]:
import os
import random

import numpy as np
import pandas as pd

import cv2 as cv
from matplotlib import pyplot as plt

# for running DeepMatching
import subprocess
from io import StringIO

In [ ]:
base_path = './some-results/'

In [ ]:
def overlay(img_1, img_2):
    
    if len(img_1.shape) == 3:
        overlay = np.zeros_like(img_1, dtype='uint8')
    else:
        overlay = np.zeros((img_1.shape[0], img_1.shape[1], 3), dtype='uint8')

    overlay[:,:,0] = img_1
    overlay[:,:,1] = img_2
    
    return overlay

## Generating some test Image Pairs

In [ ]:
camera_view = 'KL11-E1DC'

availible_ids = [int(file.split('.')[0])
                 for file in os.listdir(f"./data/real/{camera_view}/") 
                 if file.endswith(".png")]

In [ ]:
import itertools
from random import shuffle
import random

random.seed(123)
n = 500
#n = 100
img_pairs = list(itertools.combinations(availible_ids, 2))
shuffle(img_pairs)
img_pairs = img_pairs[:n]
img_pairs[:3] # check this doesn't change!

In [ ]:
# check we don't have any duplicates
for id_1, id_2 in img_pairs:
    if id_1 == id_2:
        print('oops, try again')

# Run DeepMatching

In [ ]:
# good idea to find out exactly how strict we can make these!
max_scale_factor = 1.05
max_rotation_angle = 2

def deepmatch(img_id_1, img_id_2):
    """
    Make a call to the DeepMatching algorithm and return the 
    resulting point pairs in a DataFrame
    """
    
    # build command
    cmd = f'./deepmatching/deepmatching {img_id_1} {img_id_2} '
    cmd += f'-max_scale {max_scale_factor} '
    cmd += f'-rot_range 0 {max_rotation_angle} '
    cmd += '-resize 500 500 ' # smaller images are processed much faster!
    #cmd += '-nt 16' # number of threads for faster completion
    
    # split so Popen can do it's thing correctly
    cmd = cmd.split()
    
    # open the subprocess
    process = subprocess.Popen(cmd,
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.PIPE)

    # wait for the process to terminate and decode
    out, err = process.communicate()
    out = StringIO(out.decode("utf-8"))
    df = pd.read_csv(out, sep=" ", header=None, 
                     names=['x_1','y_1','x_2','y_2','score','index'])
    
    return df


def deepmatch_many(img_pairs, data_path):
    """
    Run DM for each image pair in a set of image pairs.
    Write point pairs to csv.
    """
    
    for id_1, id_2 in img_pairs:

        img1_path = os.path.join(data_path, str(id_1) + '.png')
        img2_path = os.path.join(data_path, str(id_2) + '.png')
        
        # deepmatch
        point_pairs = deepmatch(img1_path, img2_path)
        
        # write results
        outpath = os.path.join('./some-results/', pp_batch, f'{id_1}_to_{id_2}.csv')
        point_pairs.to_csv(outpath)

In [ ]:
# name of the preprocessing stratergy to run DM on
pp_batch = 'unpp'

# build the path of where the preprocessed images are saved
data_path = os.path.join('./some-results/data/', pp_batch)

deepmatch_many(img_pairs, data_path)

# Scoring

Kept some plots of edge detection here from when looking at early ways to score. An original idea was to only score pixels that were near a detected edge (as this is where red/green fringes will be found). This might be useful in the future. Note also that the Laplacian edge detection with a large radius parameter is actually not as bad as we thought in the first week of the project.

In [ ]:
pair = img_pairs[63]

pp_path = 'clustered_norm'

img_1 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
img_2 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)

In [ ]:
plt.figure(figsize=(15,15))
for i, r in enumerate(np.linspace(5,200,9).astype(int)):
    plt.subplot(331+i)
    diff = np.abs(img_1.astype(int) - cv.blur(img_1, (r,r)).astype(int))
    plt.imshow(diff)
    plt.title(f'mean filter of size {r}')

In [ ]:
plt.figure(figsize=(15,15))
for i, r in enumerate(np.linspace(5,200,9).astype(int)):
    if r % 2 == 0:
        r += 1
    plt.subplot(331+i)
    diff = np.abs(img_1.astype(int) - cv.GaussianBlur(img_1, (r,r), 0).astype(int))
    plt.imshow(diff)
    plt.title(f'Gaussian filter of size {r}')

In [ ]:
plt.figure(figsize=(15,15))
for i, r in enumerate(np.linspace(5,31,9).astype(int)):
    if r % 2 == 0:
        r += 1
    plt.subplot(331+i)
    diff = np.abs(cv.Laplacian(img_1, cv.CV_64F, ksize=r).astype(int))
    plt.imshow(diff)
    plt.title(f'Laplacian of size {r}')

In [ ]:
r = 200
img = img_1[300:450, 210:310]

dst = cv.blur(img, (r, r))
isolated = np.abs(img.astype(int) - dst.astype(int))

plt.imshow(isolated)
plt.colorbar()

So we could only score pixels above a certain threshold here.

# Diff Diff Method

We need two images to be matched $l_1$ and $l_2$, and a transformation of the first image $l_1'$. We assess the score of the tranformation using 
$$
|l_1 - l_2| - |l_1' - l_2|
$$

In [ ]:
def crop_image(img, b=5):
    rows, cols = img.shape
    img = img.copy()
    img[0:b, :] = 0
    img[rows-b:rows, :] = 0
    img[:, 0:b] = 0
    img[:, cols-b:cols] = 0

    return img

def get_diff_diff(img_1, img_1_h, img_2, crop=0, tol=50, flat_filtered=True):
    if crop != 0:
        img_1 = crop_image(img_1, crop)
        img_1_h = crop_image(img_1_h, crop)
        img_2 = crop_image(img_2, crop)
    
    diff_before = np.abs(img_1.astype(int) - img_2.astype(int))
    diff_after = np.abs(img_1_h.astype(int) - img_2.astype(int))
    diff_diff = diff_before - diff_after
    if flat_filtered:
        diff_diff_filtered = diff_diff[np.abs(diff_diff) > tol]
        return diff_diff_filtered
    else:
        return diff_diff

    

def show_diff_diff(img_1, img_1_h, img_2, crop=0, tol=50):
    if crop != 0:
        img_1 = crop_image(img_1, crop)
        img_1_h = crop_image(img_1_h, crop)
        img_2 = crop_image(img_2, crop)
        
    dd = get_diff_diff(img_1, img_1_h, img_2, crop, tol, flat_filtered=False)
    overl = overlay(img_1_h, img_2)
    overl[np.where(dd > tol)] = 255
    overl[np.where(dd < -tol)] = 0
    plt.imshow(overl, vmin=0, vmax=255)
    
def diff_diff_hist(img_1, img_1_h, img_2, tol=50, crop=0):
    if crop != 0:
        img_1 = crop_image(img_1, crop)
        img_1_h = crop_image(img_1_h, crop)
        img_2 = crop_image(img_2, crop)
        
    counts, _, _ = plt.hist(get_diff_diff(img_1, img_1_h, img_2, crop, tol), bins=255//15, range=(-255,255) )
    plt.plot([tol-5,tol-5], [0, max(counts) * 1.1], linewidth=5, color='black')
    plt.plot([-tol+5,-tol+5], [0, max(counts) * 1.1], linewidth=5, color='black')
    
    plt.xlabel("$|l_1 - l_2| - |l_1' - l_2|$")
    plt.ylabel('Count')
    plt.show()
    
def dd_score(img_1, img_1_h, img_2, tol=50, crop=0):
    """
    TODO: if less than X are above tolerance, then return that 
    the image has not substantially changed. This is a bit tricky 
    ratio becomes unstable for low counts of changed pixel locations,
    but if the images are already aligned, then the score should not
    pick up anything.
    So: how to distingush between: bad alignment but transformation
    has changed nothing and good initial alignment so transformation
    doesn't need to change anything. (can probably do by checking if
    the score is at a minimum in homography space)
    """
    if crop != 0:
        img_1 = crop_image(img_1, crop)
        img_1_h = crop_image(img_1_h, crop)
        img_2 = crop_image(img_2, crop)
        
    dd = get_diff_diff(img_1, img_1_h, img_2, crop, tol)
    pos = np.where(dd > tol)[0]
    neg = np.where(dd < -tol)[0]
    if len(dd) == 0:
        return 0.5, 0
    #
    # instead of weighting, could just return the ratio
    # and an additional 'confidence' score based on the
    # number of points and/or sublock agreements 
    #
    #return len(pos) / len(neg) * np.log(len(dd))*2
    return len(pos) / len(dd), len(dd)
    #return len(pos) / len(neg) * len(dd)**0.1
    # ratio and count separately
    


## Full Image

In [ ]:
pair = img_pairs[63]

pp_path = 'clustered_norm'

img_1 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
img_2 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)

point_pairs = pd.read_csv(os.path.join(base_path, pp_path, str(pair[0]) + '_to_' + str(pair[1]) + '.csv'),
                         index_col=0)

before = point_pairs[['x_1','y_1']].values
after = point_pairs[['x_2','y_2']].values

h_matrix, mask = cv.findHomography(before, after, method=cv.RANSAC)
img_1_h = cv.warpPerspective(img_1, h_matrix, dsize=img_1.shape)

plt.figure(1, figsize=(17,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2))

plt.subplot(122)
plt.title(f'DM Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1_h, img_2))

Looking at the above matching, we can see that the tile section in the bottom-centre of the image are much improved, while other regions such as the bottom right are mostly unchanged. Let's plot on top of this transformed image the regions our scoring system is detecting change. We see that after filtering out some noise, we get very sensible results.

In [ ]:
plt.figure(figsize=(16,16))

for i in range(9):
    plt.subplot(331 + i)
    show_diff_diff(img_1, img_1_h, img_2, crop=5, tol=i*10+10)
    plt.title(f'tol = {i*10 + 10}')
plt.tight_layout()

In [ ]:
diff_diff_hist(img_1, img_1_h, img_2, 50, crop=5)

In [ ]:
dd_score(img_1, img_1_h, img_2, crop=5)

The histogram shows that there are more locations in the transformed pair that have a smaller distance between pixel intensities. i.e. the images are more similar

## ROI - Tiles

We can select a region where we expect to see a lot of this behaviour, to remove noise from other parts of the image.

In [ ]:
img_1_r1 = img_1[300:450, 190:310]
img_2_r1 = img_2[300:450, 190:310]
img_1_h_r1 = img_1_h[300:450, 190:310]

Take the differences of the two images at each position to get information about the red/green fringes. Do this after transforming the image as well.

In [ ]:
diff_diff_hist(img_1_r1, img_1_h_r1, img_2_r1, 50)

The skew here is evidence that the fringes have gone away

In [ ]:
plt.figure(1, figsize=(9,6))

plt.subplot(131)
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)

plt.title(f'Overlaid Images')
plt.imshow(overlay(img_1[300:450, 190:310], img_2[300:450, 190:310]))

plt.subplot(132)
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)

plt.title(f'Transformed Overlay')
plt.imshow(overlay(img_1_h[300:450, 190:310], img_2[300:450, 190:310]))

plt.subplot(133)
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)

plt.title(f'Scored Overlay')

dd = get_diff_diff(img_1_r1, img_1_h_r1, img_2_r1, 0, 50, flat_filtered=False)
overl = overlay(img_1_h_r1, img_2_r1)
overl[np.where(dd > 50)] = 255
overl[np.where(dd < -50)] = 0
plt.imshow(overl, vmin=0, vmax=255)

In [ ]:
dd_score(img_1_r1, img_1_h_r1, img_2_r1)

Regions of interest will also be useful to detect regions of the image that have been badly scored. Note instead of ROIs you could just split each image into a 2x2 grid and then score each subblock. Then if one subblock has a score which is very different you could ignore it as an outlier. An example of the kind of situaion where this is needed is

## ROI - Top right

In [ ]:
img_1_r2 = img_1[0:50, 450:500]
img_2_r2 = img_2[0:50, 450:500]
img_1_h_r2 = img_1_h[0:50, 450:500]

In [ ]:
diff_diff_hist(img_1_r2, img_1_h_r2, img_2_r2)

In [ ]:
show_diff_diff(img_1_r2, img_1_h_r2, img_2_r2)

In [ ]:
plt.figure(1, figsize=(17,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1_r2, img_2_r2))

plt.subplot(122)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1_h_r2, img_2_r2))

In [ ]:
dd_score(img_1_r2, img_1_h_r2, img_2_r2)

The method is very accurate as demonstrated here.

## ROI - Side

In [ ]:
img_1_r3 = img_1[200:490, 100:175]
img_2_r3 = img_2[200:490, 100:175]
img_1_h_r3 = img_1_h[200:490, 100:175]

plt.imshow(img_1_r3)

In [ ]:
diff_diff_hist(img_1_r3, img_1_h_r3, img_2_r3)

In [ ]:
show_diff_diff(img_1_r3, img_1_h_r3, img_2_r3)

In [ ]:
dd_score(img_1_r3, img_1_h_r3, img_2_r3)

## Negative Case

Translate img_1 in the wrong direction and check the diff diff.

In [ ]:
rows,cols = img_1.shape

dx = 10
dy = 10

M = np.float32([[1,0,dx],[0,1,dy]])
img_1_t = cv.warpAffine(img_1, M, (cols,rows))

In [ ]:
plt.figure(1, figsize=(17,9))

plt.subplot(121)
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}', fontsize=15)
plt.imshow(overlay(img_1, img_2))

plt.subplot(122)
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}', fontsize=15)
plt.imshow(overlay(img_1_t, img_2))
plt.tight_layout()

In [ ]:
diff_diff_hist(img_1, img_1_t, img_2, crop=10)

In [ ]:
plt.figure(figsize=(7,7))
show_diff_diff(img_1, img_1_t, img_2, crop=10)

In [ ]:
dd_score(img_1, img_1_t, img_2, crop=10)

As expected, the score is low. Looking at some of the specific regions:

In [ ]:
diff_diff_hist(img_1[300:450, 210:310], img_1_t[300:450, 210:310], img_2[300:450, 210:310])

In [ ]:
show_diff_diff(img_1[300:450, 210:310], img_1_t[300:450, 210:310], img_2[300:450, 210:310])

### Brute force alignment

Calculate the ddscore as a function of translation distance. Originally just wanted to show that increasing translation distance lead to a decreasing score, but actually from the peak here we can find the best alignment.

In [ ]:
xs = []
ys = []
dists = []
scores = []

for dx in range(10):
    for dy in range(1,10):
        xs.append(dx)
        ys.append(dy)
        
        dist = (dx**2 + dy**2)**0.5
        dists.append(dist)

        M = np.float32([[1,0,dx],[0,1,dy]])
        img_1_t = cv.warpAffine(img_1, M, (cols,rows))

        score = dd_score(img_1, 
                         img_1_t,
                         img_2, crop=15, tol=50)[0]
        
        scores.append(score)

plt.scatter(dists, scores)
plt.xlabel('translation distance', fontsize=14)
plt.ylabel('dd_score', fontsize=14)

In general the trend of decrasing score with increasing distance is a good one. What about looking at that peak though?

In [ ]:
best_index = np.argmax(scores)
x_best, y_best = xs[best_index], ys[best_index]
print(x_best, y_best)

M = np.float32([[1,0,x_best],[0,1,y_best]])
img_1_t_best = cv.warpAffine(img_1, M, (cols,rows))

In [ ]:
plt.figure(1, figsize=(14,9))

plt.subplot(121)
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}', fontsize=15)
plt.imshow(overlay(img_1, img_2))

plt.subplot(122)
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title(f'Brute Force Translation - DD Optimised', fontsize=15)
plt.imshow(overlay(img_1_t_best, img_2))
plt.tight_layout()

Optimising the DD score here actually gets us a better match than from deepmatching! This is not usually the case, and seems to have been lucky. But we could still entertain the idea of optimising the full homography matrix using the ddscore.

# Test New Image

We run into a fail case with the low contrast images

In [ ]:
pair = img_pairs[2]

pp_path = 'ds_bl_nlm'

img_1 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
img_2 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)

point_pairs = pd.read_csv(os.path.join(base_path, pp_path, str(pair[0]) + '_to_' + str(pair[1]) + '.csv'),
                         index_col=0)

before = point_pairs[['x_1','y_1']].values
after = point_pairs[['x_2','y_2']].values

h_matrix, mask = cv.findHomography(before, after, method=cv.RANSAC)
img_1_h = cv.warpPerspective(img_1, h_matrix, dsize=img_1.shape)

plt.figure(1, figsize=(11,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2)[0:100,400:500])

plt.subplot(122)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1_h, img_2)[0:100,400:500])

here the match in the top right is much improved. However, because of the residual red glow from img_1 which has moved into the dark region of the image, we detect a lot of badly scored pixels. An alternative to subblock scoring would be to treat the negatives separately, or just ignore them (as they can happen in this type of circumstance).

You can't really prevent this through clever choice of tolerance either.

In [ ]:
plt.figure(figsize=(16,16))

tols = np.linspace(10,30,9).astype(int)

for i in range(9):
    plt.subplot(331 + i)
    show_diff_diff(img_1, img_1_h, img_2, crop=5, tol=tols[i])
    plt.title(f'tol = {tols[i]}')
plt.tight_layout()

# Fail Case

Example of classic fail case of the scoring algorithm

In [ ]:
pair = img_pairs[6]

pp_path = 'ds_bl_nlm'

img_1 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
img_2 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)

point_pairs = pd.read_csv(os.path.join(base_path, pp_path, str(pair[0]) + '_to_' + str(pair[1]) + '.csv'),
                         index_col=0)

before = point_pairs[['x_1','y_1']].values
after = point_pairs[['x_2','y_2']].values

h_matrix, mask = cv.findHomography(before, after, method=cv.RANSAC)
img_1_h = cv.warpPerspective(img_1, h_matrix, dsize=img_1.shape)

plt.figure(1, figsize=(14,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2)[400:500,400:500], vmin=0, vmax=255)

plt.subplot(122)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1_h, img_2)[400:500,400:500], vmin=0, vmax=255)

Above we can see that the bottom right of the image has clearly improved accross the central diagonal curve (the green band on the left is small than the red band on the right). The problem is, in this case, and in this region, img_1 is substantially more bright than image 2, by about 20 pixels. (in the very bottom right, we have a red fringe replaced with a green fringe - essentially no change. the algorithm successfully handles this case). We can note the difference from the overall reddy tinge to the overlay.

In [ ]:
print(img_1[400:500,400:500].mean())
print(img_2[400:500,400:500].mean())

In [ ]:
plt.figure(1, figsize=(10,9))

plt.subplot(121)
plt.title(f'img_1 - brighter overall')
plt.imshow(img_1[400:500,400:500], vmin=0, vmax=255)

plt.subplot(122)
plt.title(f'img_2 - darker overall')
plt.imshow(img_2[400:500,400:500], vmin=0, vmax=255)

The specific combination of a bright fringe in the top right of img_1 and the very dark patch in the middle right on img_2 means that when that bright patch slightly intrudes on teh dark patch we get a big swing. 

We can see that for any choice of tolerance, we detect this region as having gotten worse.

In [ ]:
plt.figure(figsize=(10,10))
tols = np.linspace(10,40,9).astype(int)

for i in range(9):
    plt.subplot(331 + i)
    show_diff_diff(img_1[400:500,400:500], img_1_h[400:500,400:500], img_2[400:500,400:500], crop=0, tol=tols[i])
    plt.title(f'tol = {tols[i]}')
plt.tight_layout()

After compensating for the lack of luminance in img_2, we restore the correct behaviour of the score.

In [ ]:
img_2[425:475,425:475] += 25

In [ ]:
plt.figure(figsize=(10,10))

tols = np.linspace(10,40,9).astype(int)

for i in range(9):
    plt.subplot(331 + i)
    show_diff_diff(img_1[400:500,400:500], img_1_h[400:500,400:500], img_2[400:500,400:500], crop=0, tol=tols[i])
    plt.title(f'tol = {tols[i]}')
plt.tight_layout()

What does this mean:
- it may be preferrable to have highly contrast corrected images, as these might have less of these absolute brightness differences (haven't confirmed this yet but it makes sense).
- it may a good idea to score the image in subblocks, so that a) we can check for subblock consistency (i.e. this bottom right subblock would stick out when all the other subblocks would be unchanged or have improved) AND/OR b) check for and correct all sizeable differences in meanillumination between subblocks.

Seems like correcting the average illumination in the subblock is feasbile: here is another very extreme example:

In [ ]:
pair = img_pairs[8]

pp_path = 'ds_bl_nlm'

img_1 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
img_2 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)

point_pairs = pd.read_csv(os.path.join(base_path, pp_path, str(pair[0]) + '_to_' + str(pair[1]) + '.csv'),
                         index_col=0)

before = point_pairs[['x_1','y_1']].values
after = point_pairs[['x_2','y_2']].values

h_matrix, mask = cv.findHomography(before, after, method=cv.RANSAC)
img_1_h = cv.warpPerspective(img_1, h_matrix, dsize=img_1.shape)

plt.figure(1, figsize=(14,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2), vmin=0, vmax=255)

plt.subplot(122)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1_h, img_2), vmin=0, vmax=255)

It's obvious from the green hue in the bottom right we have crazy mismatched brightness in that region.

In [ ]:
img_1[400:500,400:500].mean() - img_2[400:500,400:500].mean()

In [ ]:
plt.figure(1, figsize=(14,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2)[400:500,400:500], vmin=0, vmax=255)

plt.subplot(122)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1_h, img_2)[400:500,400:500], vmin=0, vmax=255)

Zooming in we don't notice any obvious difference that the homography has made. However the score picks up a lot of signal! The nice thing is we show that it actually should be detecting SOMETHING here, it just gets confused by the relative illumination.

In [ ]:
plt.figure(figsize=(10,10))
tols = np.linspace(5,20,9).astype(int)

for i in range(9):
    plt.subplot(331 + i)
    show_diff_diff(img_1[400:500,400:500], img_1_h[400:500,400:500], img_2[400:500,400:500], crop=0, tol=tols[i])
    plt.title(f'tol = {tols[i]}')
plt.tight_layout()

Let's correct that change in luminance.

In [ ]:
img_2[300:500,300:500] -=100


plt.figure(1, figsize=(17,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2)[300:500,300:500], vmin=0, vmax=255)

plt.subplot(122)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1_h, img_2)[300:500,300:500], vmin=0, vmax=255)

It's now mch more obvious that the homography has actually gotten worse from the (still quite faint) red  fringes in the right hand image. After applying that correction we see expected behaviour from the scoring.

In [ ]:
plt.figure(figsize=(16,16))

tols = np.linspace(5,20,9).astype(int)

for i in range(9):
    plt.subplot(331 + i)
    show_diff_diff(img_1, img_1_h, img_2, crop=5, tol=tols[i])
    plt.title(f'tol = {tols[i]}')
plt.tight_layout()

# A New Image

In [ ]:
pair = img_pairs[12]

pp_path = 'clustered_norm'

img_1 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
img_2 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)

point_pairs = pd.read_csv(os.path.join(base_path, pp_path, str(pair[0]) + '_to_' + str(pair[1]) + '.csv'),
                         index_col=0)

before = point_pairs[['x_1','y_1']].values
after = point_pairs[['x_2','y_2']].values

h_matrix, mask = cv.findHomography(before, after, method=cv.RANSAC)
img_1_h = cv.warpPerspective(img_1, h_matrix, dsize=img_1.shape)

plt.figure(1, figsize=(17,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2))


plt.subplot(122)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1_h, img_2))
plt.savefig(os.path.join(base_path, 'homographies', pp_path, str(pair[0]) + '_to_' + str(pair[1]) + '.png'),
           dpi=300)

In [ ]:
diff_diff_hist(img_1, img_1_h, img_2, crop=5)

In [ ]:
plt.figure(figsize=(6,6))
show_diff_diff(img_1, img_1_h, img_2, crop=5)

We are picking up exactly the right things.

In [ ]:
dd_score(img_1, img_1_h, img_2, crop=5)

In [ ]:
def brute_force_tf(img_1, img_2, tol=50):
    
    rows, cols = img_1.shape
    x_min, y_min = int(rows*0.05), int(cols*0.05)
    x_max, y_max = int(rows*0.95), int(cols*0.95)

    xs = []
    ys = []
    dists = []
    scores = []

    for dx in range(15):
        for dy in range(1,15):
            xs.append(dx)
            ys.append(dy)

            dist = (dx**2 + dy**2)**0.5
            dists.append(dist)

            M = np.float32([[1,0,dx],[0,1,dy]])
            img_1_t = cv.warpAffine(img_1, M, (cols,rows))

            score = dd_score(img_1[y_min:y_max, x_min:x_max], 
                             img_1_t[y_min:y_max, x_min:x_max],
                             img_2[y_min:y_max, x_min:x_max], tol=tol)

            scores.append(score)
            
    return xs, ys, dists, scores

In [ ]:
xs, ys, dists, scores = brute_force_tf(img_1, img_2, 40)

In [ ]:
plt.scatter(dists, scores)
plt.xlabel('translation distance')
plt.ylabel('dd_score')

In [ ]:
best_index = np.argmax(scores)
x_best, y_best = xs[best_index], ys[best_index]
print(x_best, y_best)

M = np.float32([[1,0,x_best],[0,1,y_best]])
img_1_t_best = cv.warpAffine(img_1, M, (cols,rows))

In [ ]:
plt.figure(1, figsize=(17,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2))

plt.subplot(122)
plt.title(f'Brute Force Translation')
plt.imshow(overlay(img_1_t_best, img_2))

In [ ]:
plt.figure(figsize=(8,8))
show_diff_diff(img_1, img_1_t_best, img_2)

Interestingly the best brute force translation is not sensitive to the tolerance. Seems like a good thing. But translation doesn't work in this case. We could definitely optimise a homography to minimise this score though (non brute force)

In [ ]:
tols = range(10,30,1)
scores = [dd_score(img_1[10:,0:490], img_1_h[10:,0:490], img_2[10:,0:490], tol=tol) for tol in tols]
plt.plot(tols, scores)

## Different pp

In [ ]:
pair = img_pairs[12]

pp_path = 'ds_bl'

img_1 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
img_2 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)

point_pairs = pd.read_csv(os.path.join(base_path, pp_path, str(pair[0]) + '_to_' + str(pair[1]) + '.csv'),
                         index_col=0)

before = point_pairs[['x_1','y_1']].values
after = point_pairs[['x_2','y_2']].values

h_matrix, mask = cv.findHomography(before, after, method=cv.RANSAC)
img_1_h = cv.warpPerspective(img_1, h_matrix, dsize=img_1.shape)

plt.figure(1, figsize=(17,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2))


plt.subplot(122)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1_h, img_2))

In [ ]:
plt.figure(figsize=(16,16))

for i in range(9):
    plt.subplot(331 + i)
    tol = i*5 + 5
    show_diff_diff(img_1, img_1_h, img_2, crop=10, tol=tol)
    plt.title(f'tol = {tol}')
plt.tight_layout()

The tolerance range for this less contrasty image is noticably different from that of the highly contrast corrected images.

In [ ]:
diff_diff_hist(img_1, img_1_h, img_2, crop=10, tol=20)

In [ ]:
plt.figure(figsize=(7,7))
show_diff_diff(img_1, img_1_h, img_2, crop=10, tol=20)

We are picking up mostly the right things.

In [ ]:
dd_score(img_1, img_1_h, img_2, crop=10, tol=20)

In [ ]:
tols = range(10,30,1)
scores = [dd_score(img_1, img_1_h, img_2, crop=10, tol=tol) for tol in tols]
plt.plot(tols, scores)

Clearly where preprocessing has increased the contrast less, we will need to adjust the tolterances. But, this is only one parameter, and also we have shown that the tolerance will scale the score distribution, but should not affect it's shape too much, as long as we are in the right kind of ballpark.

# Closer Look At Tolerance

We have seen tolerance depends on the contrast in the images, so we should be able to detect the contrast and set the tolerance.

In [ ]:
pair = img_pairs[12]

pp_path = 'ds_bl'

img_1_soft = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
img_2_soft = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)

pp_path = 'clustered_norm'

img_1_hard = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
img_2_hard = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)

These images are obviously visually very different.

In [ ]:
plt.subplot(121)
plt.imshow(img_1_soft, vmin=0, vmax=255)
plt.subplot(122)
plt.imshow(img_1_hard, vmin=0, vmax=255)

In [ ]:
print(img_1_soft.std(), img_1_hard.std())
print(img_1_soft.mean(), img_1_hard.mean())

In [ ]:
print(img_2_soft.std(), img_2_hard.std())
print(img_2_soft.mean(), img_2_hard.mean())

However the metrics we have been using previously are completely unchanged! i.e. to our clustering etc algorithms there is no change between the images! So what is actually different about these two images? - _Local Contrast_

In [ ]:
stds = []
means = []
meds = []

for pair in img_pairs:
    pp_path = 'ds_bl'

    img_1_soft = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
    img_2_soft = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)

    pp_path = 'clustered_norm'

    img_1_hard = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
    img_2_hard = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)
    
    std_1 = img_1_soft.std() - img_1_hard.std()
    std_2 = img_2_soft.std() - img_2_hard.std()
    mean_1 = img_1_soft.mean() - img_1_hard.mean()
    mean_2 = img_2_soft.mean() - img_2_hard.mean()
    med_1 = np.median(img_1_soft) - np.median(img_1_hard)
    med_2 = np.median(img_2_soft) - np.median(img_2_hard)
    
    stds.append([std_1, std_2])
    means.append([mean_1, mean_2])
    meds.append([med_1, med_2])
    
stds = np.array(stds)
means = np.array(means)
meds = np.array(meds)

In [ ]:
print(stds.mean(), stds.std())
print(means.mean(), means.std())
print(meds.mean(), meds.std())

We need to look at local calculations of the above quantities.

In [ ]:
# Define the window size
def get_local_contrast(img, windowsize_r = 50, windowsize_c = 50):
    stds = []
    means = []

    # crop out the window 
    for r in range(0,img.shape[0] - windowsize_r, windowsize_r):
        for c in range(0,img.shape[1] - windowsize_c, windowsize_c):
            window = img[r:r+windowsize_r,c:c+windowsize_c]
            means.append(window.mean())
            stds.append(window.std())

    mean = np.array(means).mean(), np.array(means).std()
    std = np.array(stds).mean(), np.array(stds).std()
    
    return mean, std

In [ ]:
mean, std = get_local_contrast(img_1_hard)
print(mean)
print(std)

In [ ]:
mean, std = get_local_contrast(img_1_soft)
print(mean)
print(std)

In [ ]:
pp_1 = []
pp_2 = []

for id_ in availible_ids:
    pp_path = 'ds_bl'

    img_1 = cv.imread(os.path.join(base_path, 'data', pp_path, str(id_) + '.png'), cv.IMREAD_GRAYSCALE)

    pp_path = 'clustered_norm'

    img_2 = cv.imread(os.path.join(base_path, 'data', pp_path, str(id_) + '.png'), cv.IMREAD_GRAYSCALE)
    
    mean_1, std_1 = get_local_contrast(img_1)
    mean_2, std_2 = get_local_contrast(img_2)
    
    pp_1.append([mean_1, std_1])
    pp_2.append([mean_2, std_2])
    
pp_1 = np.array(pp_1)
pp_2 = np.array(pp_2)

In [ ]:
# mean of each subblock
mean_mean_mean = np.round(pp_1[:,0,0].mean(), 2)
mean_std_mean = np.round(pp_1[:,0,1].mean(), 2)
std_mean_mean = np.round(pp_1[:,0,0].std(), 2)
std_std_mean = np.round(pp_1[:,0,1].std(), 2)

# variance of each subblock
mean_mean_std = np.round(pp_1[:,1,0].mean(), 2)
mean_std_std = np.round(pp_1[:,1,1].mean(), 2)
std_mean_std = np.round(pp_1[:,1,0].std(), 2)
std_std_std = np.round(pp_1[:,1,1].std(), 2)

print('LOW CLAHE PP')
print('Distribution (over images) of mean (over subblocks) of mean (over intinsities)')
print(mean_mean_mean, '+\-', std_mean_mean)
print('(average pixel intensity accross all images)')

print('\nDistribution (over images) of mean (over subblocks) of std (over intinsities)')
print(mean_mean_std, '+\-', std_mean_std)
print('(on average, subblocks have less contrast)')

print('\nDistribution (over images) of std (over subblocks) of mean (over intinsities)')
print(mean_std_mean, '+\-', std_std_mean)
print('(average subblock intensity is more variable)')

print('\nDistribution (over images) of std (over subblocks) of std (over intinsities)')
print(mean_std_std, '+\-', std_std_std)
print('??')

In [ ]:
# mean of each subblock
mean_mean_mean = np.round(pp_2[:,0,0].mean(), 2)
mean_std_mean = np.round(pp_2[:,0,1].mean(), 2)
std_mean_mean = np.round(pp_2[:,0,0].std(), 2)
std_std_mean = np.round(pp_2[:,0,1].std(), 2)

# variance of each subblock
mean_mean_std = np.round(pp_2[:,1,0].mean(), 2)
mean_std_std = np.round(pp_2[:,1,1].mean(), 2)
std_mean_std = np.round(pp_2[:,1,0].std(), 2)
std_std_std = np.round(pp_2[:,1,1].std(), 2)

print('HIGH CLAHE PP')
print('Distribution (over images) of mean (over subblocks) of mean (over intinsities)')
print(mean_mean_mean, '+\-', std_mean_mean)
print('(average pixel intensity accross all images)')

print('\nDistribution (over images) of mean (over subblocks) of std (over intinsities)')
print(mean_mean_std, '+\-', std_mean_std)
print('(on average, subblocks have more contrast)')

print('\nDistribution (over images) of std (over subblocks) of mean (over intinsities)')
print(mean_std_mean, '+\-', std_std_mean)
print('(average subblock intensity is less variable)')

print('\nDistribution (over images) of std (over subblocks) of std (over intinsities)')
print(mean_std_std, '+\-', std_std_std)
print('??')

There is a new metric in some of these we can use. We can probably automatically set the tolerance based on how much CLAHE has been performed, which will be easy to find out using these new metrics.

ALSO you could calculate the contrast properties of each image, and then have a asymmetric tolerance for changes for each such that you can match images whose contrasts are not the similar. This could then be done at the subblock level, which hopefully would mitigate the fail cases described above.

In terms of preprocessing, from the way that this new score is calculated (i.e. reliant on per pixel changes) it makes sense that noise in the image would be our biggest enemy, especially if we can get some resilience to different contrasts through 

# Auto Tolerance

Initial assumption that all images will have similar amounts of pp, i.e. we are determining the tolerance for the entire imgbank (to remove differences between very light and very harsh pp). We could make this per image.

In [ ]:
# Define the window size
def get_local_contrast(img, windowsize_r = 50, windowsize_c = 50):
    stds = []
    means = []

    # crop out the window 
    for r in range(0,img.shape[0] - windowsize_r, windowsize_r):
        for c in range(0,img.shape[1] - windowsize_c, windowsize_c):
            window = img[r:r+windowsize_r,c:c+windowsize_c]
            means.append(window.mean())
            stds.append(window.std())

    mean = np.array(means).mean(), np.array(means).std()
    std = np.array(stds).mean(), np.array(stds).std()
    
    mean_subblock_contrast = std[0]
    variance_between_subblocks = mean[1]
    std_std = std[1]
    
    return mean_subblock_contrast, variance_between_subblocks, std_std

In [ ]:
pair = img_pairs[12]

pp_path = 'clustered_norm'

img_1 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
img_2 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)

point_pairs = pd.read_csv(os.path.join(base_path, pp_path, str(pair[0]) + '_to_' + str(pair[1]) + '.csv'),
                         index_col=0)

before = point_pairs[['x_1','y_1']].values
after = point_pairs[['x_2','y_2']].values

h_matrix, mask = cv.findHomography(before, after, method=cv.RANSAC)
img_1_h = cv.warpPerspective(img_1, h_matrix, dsize=img_1.shape)



plt.figure(1, figsize=(17,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2)[300:500,300:500], vmin=0, vmax=255)

plt.subplot(122)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1_h, img_2)[300:500,300:500], vmin=0, vmax=255)

The stratergy will be to try several images from the same pp bundle and measure tolerance and local contrast for each. Then compare between different bundles.

In [ ]:
img_1[300:500,300:500].mean() - img_2[300:500,300:500].mean()

In [ ]:
plt.figure(figsize=(16,16))

tols = np.linspace(10,50,9).astype(int)

for i in range(9):
    plt.subplot(331 + i)
    show_diff_diff(img_1, img_1_h, img_2, crop=10, tol=tols[i])
    plt.title(f'tol = {tols[i]}')
plt.tight_layout()

In [ ]:
dd_score(img_1, img_1_h, img_2, crop=10, tol=45)

In [ ]:
np.round(np.array([get_local_contrast(img_1),
          get_local_contrast(img_1_h),
          get_local_contrast(img_2)]).mean(axis=0), 2)

After looking through several cases, I numerically fit the tolerance to using a linear function.

In [ ]:
def get_tol(img):
    mean_subblock_contrast = get_local_contrast(img)[0]
    return mean_subblock_contrast*1.12 + 1.88

# Score Normalisation

We don't want our score to be dependend on the contrast in the images. Different image contrasts necessisstate different tolerances, and the differing contrast and the change in tolerance can both change the absolute value of the score. What we can do is take a matched trio of images and count the score for a range of different contrast on this match. Then we can deduce how the scores depend on the contrast in the images, and correct for that dependence.

Start out by doing the same CC to all three images, but could probably generalise to have different CC to each image in the match set.

[this is unfinished, instead we fixed scoring preprocessing independently of matching preprocessing]

In [ ]:
def correct_contrast(img, clip_limit, n_tiles_per_row):
    
    # note that the actual affects of the clip limit 
    # depend on the normalisation of the histogram
    # and therefore the number of tiles in the grid
    
    # create the object
    clahe = cv.createCLAHE(clipLimit=clip_limit, 
                           tileGridSize=(n_tiles_per_row, n_tiles_per_row))
    
    # apply equalisation
    return clahe.apply(img)

In [ ]:
pair = img_pairs[12]

pp_path = 'ds_bl_nlm'

img_1 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
img_2 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)

point_pairs = pd.read_csv(os.path.join(base_path, pp_path, str(pair[0]) + '_to_' + str(pair[1]) + '.csv'),
                         index_col=0)

before = point_pairs[['x_1','y_1']].values
after = point_pairs[['x_2','y_2']].values

h_matrix, mask = cv.findHomography(before, after, method=cv.RANSAC)
img_1_h = cv.warpPerspective(img_1, h_matrix, dsize=img_1.shape)

plt.figure(1, figsize=(10,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2))

plt.subplot(122)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1_h, img_2))

Frist let's manually find the app

In [ ]:
plt.figure(figsize=(16,16))

for i in range(9):
    plt.subplot(331 + i)
    show_diff_diff(img_1, img_1_h, img_2, crop=5, tol=i*10+10)
    plt.title(f'tol = {i*10 + 10}')
plt.tight_layout()

In [ ]:
clips = np.linspace(0.00000001, 10, 20)

img_1s = [correct_contrast(img_1, clip, 20) for clip in clips]
img_1_hs = [correct_contrast(img_1_h, clip, 20) for clip in clips]
img_2s = [correct_contrast(img_2, clip, 20) for clip in clips]

In [ ]:
tols = [np.array([get_tol(img_1), get_tol(img_1_h), get_tol(img_2)]).mean() 
        for img_1, img_1_h, img_2 in zip(img_1s, img_1_hs, img_2s)]

In [ ]:
[dd_score(img_1, img_1_h, img_2, tol=tol) for img_1, img_1_h, img_2, tol in zip(img_1s, img_1_hs, img_2s, tols)]

# Noise removal

Notive a lot of salt and pepper noise on the score from dirt in the optics and other sources of noise.

In [ ]:
pair = img_pairs[12]

pp_path = 'ds_bl_nlm'

img_1 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
img_2 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)

point_pairs = pd.read_csv(os.path.join(base_path, pp_path, str(pair[0]) + '_to_' + str(pair[1]) + '.csv'),
                         index_col=0)

before = point_pairs[['x_1','y_1']].values
after = point_pairs[['x_2','y_2']].values

h_matrix, mask = cv.findHomography(before, after, method=cv.RANSAC)
img_1_h = cv.warpPerspective(img_1, h_matrix, dsize=img_1.shape)

show_diff_diff(img_1, img_1_h, img_2, tol=9, crop=5)

We can reduce this by applying a median filter to the scoring layer of the image.

In [ ]:
tol = 7

img_1 = crop_image(img_1)
img_1_h = crop_image(img_1_h)
img_2 = crop_image(img_2)

diff_before = np.abs(img_1.astype(int) - img_2.astype(int))
diff_after = np.abs(img_1_h.astype(int) - img_2.astype(int))
diff_diff = diff_before - diff_after

better = np.zeros_like(diff_diff)
worse = np.zeros_like(diff_diff)

better[diff_diff > tol] = 1
worse[diff_diff < -tol] = 1

better_med_3 = cv.medianBlur(better.astype('uint8'),3)
worse_med_3 = cv.medianBlur(worse.astype('uint8'),3)

better_med_5 = cv.medianBlur(better.astype('uint8'),5)
worse_med_5 = cv.medianBlur(worse.astype('uint8'),5)

plt.figure(figsize=(17,10))

plt.subplot(131)
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title('raw score')
img_1_c = img_1.copy()
img_1_c[better == 1] = 255
img_1_c[worse == 1] = 0
plt.imshow(img_1_c)

plt.subplot(132)
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title('median blur r=3')
img_1_c = img_1.copy()
img_1_c[better_med_3 == 1] = 255
img_1_c[worse_med_3 == 1] = 0
plt.imshow(img_1_c)

plt.subplot(133)
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title('median blur r=5')
img_1_c = img_1.copy()
img_1_c[better_med_5 == 1] = 255
img_1_c[worse_med_5 == 1] = 0
plt.tight_layout()
plt.imshow(img_1_c)

This does come at the cost of some reduced signal, but in the `r=3` case signal lossess are minimal and noise is dramatically supressed

# Score with Subblocks

In [ ]:
def ddscore_sub(img_1, img_1_h, img_2, tol, crop):
    """
    understand better the behaviour of subtracting pixel
    intensities: what happens of we go outside [0,255]?
    (at the moment we are just converting to int and then
    forgetting about it)
    """

    #crop = 10
    newlen = img_1.shape[0] - 2*crop

    windowsize_r = 96
    windowsize_c = 96   
    
    windowsize_r = newlen // 5
    windowsize_c = newlen // 5   
    
    img_1_COPY = img_1.copy()
    img_1_h_COPY = img_1_h.copy()
    img_2_COPY = img_2.copy()
    
    img_1 = crop_image(img_1, b=crop).astype(int)[crop:img_1.shape[0]-crop,crop:img_1.shape[0]-crop]
    img_1_h = crop_image(img_1_h, b=crop).astype(int)[crop:img_1_h.shape[0]-crop,crop:img_1_h.shape[0]-crop]
    img_2 = crop_image(img_2, b=crop).astype(int)[crop:img_2.shape[0]-crop,crop:img_2.shape[0]-crop]
    scores = []
    counts = []

    # crop out the window 
    for r in range(0,img_1.shape[0] - windowsize_r + 1, windowsize_r):
        for c in range(0,img_1.shape[1] - windowsize_c + 1, windowsize_c):
            #print(r,c)
            img_1_sb = img_1[r:r+windowsize_r,c:c+windowsize_c]
            img_1_h_sb = img_1_h[r:r+windowsize_r,c:c+windowsize_c]
            img_2_sb = img_2[r:r+windowsize_r,c:c+windowsize_c]
            
            mean_diff = img_2_sb.mean() - img_1_sb.mean()
            img_2_sb -= int(np.round(mean_diff))
            img_2_sb = np.clip(img_2_sb, 0, 255)
            
            score, count = dd_score(img_1_sb, img_1_h_sb, img_2_sb, tol=tol, crop=0)
            scores.append(score)
            counts.append(count)
    
    scores = np.array(scores)
    counts = np.array(counts)
    debug=False
    if debug:
        print("I HAVE A NEW IMAGE")
        print(scores)
        print(counts)
        show_dd_sb(img_1_COPY, img_1_h_COPY, img_2_COPY, tol=tol, crop=crop)
    

    if counts.sum() < 100:
        #print('none')
        return 0.5,0
    
    scores[counts < 20] = 0.5
    counts[counts < 20] = 0
    #scores = scores[counts > 50]
    #print(scores)
    #if len(scores) == 0:
    #   print('none')
    #    return 0.5,0

    
    
    
    if counts.sum() == 0:
        #print('none')
        return 0.5,0
    if debug:
        print(scores)
        print(counts)
        print(np.average(scores,weights=counts))
    return np.average(scores,weights=counts), counts.sum()



In [ ]:
pair = img_pairs[12]

pp_path = 'clustered_norm'

img_1 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
img_2 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)


point_pairs = pd.read_csv(os.path.join(base_path, pp_path, str(pair[0]) + '_to_' + str(pair[1]) + '.csv'),
                         index_col=0)

before = point_pairs[['x_1','y_1']].values
after = point_pairs[['x_2','y_2']].values

h_matrix, mask = cv.findHomography(before, after, method=cv.RANSAC)
img_1_h = cv.warpPerspective(img_1, h_matrix, dsize=img_1.shape)


plt.figure(1, figsize=(17,4))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2)[300:500,300:500], vmin=0, vmax=255)

plt.subplot(122)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1_h, img_2)[300:500,300:500], vmin=0, vmax=255)

In [ ]:
def show_dd_sb(img_1, img_1_h, img_2, tol, crop):
    """
    Visualisation for the subblock scoring.
    Much better implementations are probably possible.
    """
    
    newlen = img_1.shape[0] - 2*crop
    
    windowsize_r = newlen // 5
    windowsize_c = newlen // 5   
    
    img_1 = crop_image(img_1, b=crop).astype(int)[crop:img_1.shape[0]-crop,crop:img_1.shape[0]-crop]
    img_1_h = crop_image(img_1_h, b=crop).astype(int)[crop:img_1_h.shape[0]-crop,crop:img_1_h.shape[0]-crop]
    img_2 = crop_image(img_2, b=crop).astype(int)[crop:img_2.shape[0]-crop,crop:img_2.shape[0]-crop]
    
    plt.figure(figsize=(10,6))
    plt.subplot(121)
    plt.imshow(overlay(img_1, img_2), vmin=0, vmax=255)
    plt.subplot(122)
    plt.imshow(overlay(img_1_h, img_2), vmin=0, vmax=255)
    plt.show()

    figa, axs = plt.subplots(5,5, figsize=(7,7), sharey=True, sharex=True)
    figa.subplots_adjust(wspace=-0.1, hspace=0)

    for i, r in enumerate(range(0,img_1.shape[0] - windowsize_r+1, windowsize_r)):
        for j, c in enumerate(range(0,img_1.shape[1] - windowsize_c+1, windowsize_c)):

            img_1_sb = img_1[r:r+windowsize_r,c:c+windowsize_c]
            img_1_h_sb = img_1_h[r:r+windowsize_r,c:c+windowsize_c]
            img_2_sb = img_2[r:r+windowsize_r,c:c+windowsize_c]

            mean_diff = img_2_sb.mean() - img_1_sb.mean()
            img_2_sb -= int(np.round(mean_diff))
            img_2_sb = np.clip(img_2_sb, 0, 255)
            
            dd = get_diff_diff(img_1_sb, img_1_h_sb, img_2_sb, crop=0, tol=tol, flat_filtered=False)
            overl = overlay(img_1_sb, img_2_sb)
            overl[np.where(dd > tol)] = 255
            overl[np.where(dd < -tol)] = 0
            axs[i,j].imshow(overl, vmin=0, vmax=255)

    plt.show()
    
show_dd_sb(img_1.astype(int), img_1_h.astype(int), img_2.astype(int), tol=30, crop=5)

# Scoring PP with DDScore

<img src="https://i.imgur.com/vdi6TKt.png" alt="drawing" width="500"/>

In [ ]:
# methods we have run deepmatching for that are to be scored
pp_methods = ['unpp', 'ds', 'clustered_norm', 'ds_cc_ds',
              'ds_cc_bl', 'ds_bl', 'ds_cc_bl_lite', 'ds_cc_bl_nlm',
              'ds_bl_nlm', 'ds_cc_bl_nlm_heavy']

In [ ]:
def score_pp(pp_method, score_path, tol):
    """
    Globally score all images preprocessed in the same way
    that are found in the path specified by pp_method. Using
    scoring preprocessing from score_path.
    """

    pp_img_path = os.path.join(base_path, 'data', pp_method)
    hom_path = os.path.join(base_path, pp_method)
    
    scores = []
    counts = []

    for img_pair in img_pairs:

        img_1_id, img_2_id = img_pair

        img_1 = cv.imread(os.path.join(score_path, str(img_1_id) + '.png'), cv.IMREAD_GRAYSCALE)
        img_2 = cv.imread(os.path.join(score_path, str(img_2_id) + '.png'), cv.IMREAD_GRAYSCALE)

        point_pairs = pd.read_csv(os.path.join(hom_path, str(img_1_id) + '_to_' + str(img_2_id) + '.csv'),
                                 index_col=0)

        before = point_pairs[['x_1','y_1']].values
        after = point_pairs[['x_2','y_2']].values

        h_matrix, mask = cv.findHomography(before, after, method=cv.RANSAC)
        img_1_h = cv.warpPerspective(img_1, h_matrix, dsize=img_1.shape)

        score, count = dd_score(img_1, img_1_h, img_2, tol=tol, crop=7)

        scores.append(score)
        counts.append(count)


    scores = np.array(scores)
    counts = np.array(counts)
    
    mean_count = counts.mean()
    
    frac_unscored = len(counts[counts == 0]) / len(scores)

    changed_score_mean = scores[scores != -1].mean()
    changed_score_std = scores[scores != -1].std()

    scores[scores == -1] = 0.5
    scores[counts < 50] = 0.5
    score_mean = scores.mean()
    score_std = scores.std()
    
    return  [mean_count, frac_unscored, changed_score_mean, 
             changed_score_std,score_mean, 
             score_std]



def score_pp_sb(pp_method, score_path, tol, crop):
    """
    Score images by breaking them up into subblocks
    """
    print(pp_method)
    pp_img_path = os.path.join(base_path, 'data', pp_method)
    hom_path = os.path.join(base_path, pp_method)
    
    best_pair = 0
    best_score = 0
    best_index = -1
    worst_pair = 0
    worst_score = 1
    worst_index=-1
    
    scores = []
    counts = []

    for i, img_pair in enumerate(img_pairs):
        #print("IMAGE PAIR")
        #print(img_pair)
        img_1_id, img_2_id = img_pair

        img_1 = cv.imread(os.path.join(score_path, str(img_1_id) + '.png'), cv.IMREAD_GRAYSCALE)
        img_2 = cv.imread(os.path.join(score_path, str(img_2_id) + '.png'), cv.IMREAD_GRAYSCALE)

        point_pairs = pd.read_csv(os.path.join(hom_path, str(img_1_id) + '_to_' + str(img_2_id) + '.csv'),
                                 index_col=0)

        before = point_pairs[['x_1','y_1']].values
        after = point_pairs[['x_2','y_2']].values

        h_matrix, mask = cv.findHomography(before, after, method=cv.RANSAC)
        img_1_h = cv.warpPerspective(img_1, h_matrix, dsize=img_1.shape)

        score, count = ddscore_sub(img_1, img_1_h, img_2, tol=tol, crop=crop)
        
        #if score > 0.5:
            #print(i)
            #print(score)
        if score > best_score and count > 1:
            #print(i)
            best_score = score
            best_pair = img_pair
            best_index = i
        if score < worst_score and count > 1:
            worst_score = score
            worst_pair = img_pair
            worst_index =i

        scores.append(score)
        counts.append(count)

    print('best')
    print(best_score)
    print(best_index, best_pair)
    
    print('worst')
    print(worst_score)
    print(worst_index, worst_pair)
    
    scores = np.array(scores)
    counts = np.array(counts)
    
    mean_count = counts.mean()
    
    frac_unscored = len(counts[counts == 0]) / len(scores)

    changed_score_mean = scores[scores != 0.5].mean()
    changed_score_std = scores[scores != 0.5].std()

    scores[scores == 0.5] = 0.5
    score_mean = scores.mean()
    score_std = scores.std()
    
    num_better = len(scores[scores > 0.5])
    num_worse = len(scores[scores < 0.5])
    num_unscored = len(scores[scores == 0.5])
    mean_better = scores[scores > 0.5].mean()
    mean_worse = scores[scores < 0.5].mean()
    std_better = scores[scores > 0.5].std()
    std_worse = scores[scores < 0.5].std()
    
    return  [mean_count, frac_unscored, changed_score_mean, 
             changed_score_std,score_mean, score_std,
             num_better, num_worse, num_unscored, 
             mean_better, std_better,
             mean_worse, std_worse]


## subblock scoring

thoughts:
- don't fully understand behaviour above, more detailed look needed (but no time). Why scores less than 0.5? (when some are above 0.5)
- still looks like dependence of score on base images
- maybe can improve stds with more pairs
- what to do with unscored images (these squash the means, making it harder to differentiate different pp)
- need to rereun clustering and pipeline with new metrics
- Possible final result for report: more pairs, comparison of unpp/clustered/pipeline - show improvement from unpp, more work needed to differentiate between differet pp, both are simiarly good
- separate out images that have significantly moved!!!
- normalise/regularise score with simulation

plan:
1. redo top pp methods
2. more image pairs
3. separate out images iwth significant changes
4. show fixed examples and failc cases - try and explain which are fixed, why that many are fixed (i.e. many pairs do not change significantly)

### Viewing Interesting Results

In [ ]:
#pair = img_pairs[174]
#pair = img_pairs[81]
#pair = img_pairs[121]
#pair = img_pairs[244]
#pair = img_pairs[292]
#pair = img_pairs[121]

# unpp test best
pair = img_pairs[42]
pair = img_pairs[46]
pair = img_pairs[6]

# worst unpp
#pair = img_pairs[95]

#pp_path = 'final-superheavy'
pp_path = 'final-unpp'
img_1 = cv.imread(os.path.join(base_path, 'data', 'clustered_norm', str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
img_2 = cv.imread(os.path.join(base_path, 'data', 'clustered_norm', str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)


point_pairs = pd.read_csv(os.path.join(base_path, pp_path, str(pair[0]) + '_to_' + str(pair[1]) + '.csv'),
                         index_col=0)

before = point_pairs[['x_1','y_1']].values
after = point_pairs[['x_2','y_2']].values

h_matrix, mask = cv.findHomography(before, after, method=cv.RANSAC)
img_1_h = cv.warpPerspective(img_1, h_matrix, dsize=img_1.shape)


'''plt.figure(1, figsize=(12,8))

plt.subplot(121)
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title(f'Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2), vmin=0, vmax=255)

plt.subplot(122)
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1_h, img_2), vmin=0, vmax=255)'''

In [ ]:
show_dd_sb(img_1.astype(int), img_1_h.astype(int), img_2.astype(int), tol=50, crop=20)

- what to do aboutserious problem with cropping leading to bad scores? just crop everything alot

### Final Scoring

In [ ]:
pp_methods = ['unpp', 'final-ds', 'final-light', 'final-heavy', 'final-superheavy']

score_base = 'final-superheavy'
score_path = os.path.join(base_path, 'data', score_base)

ddscore_df = pd.DataFrame(columns=['mean_count', 'frac_unscored', 'changed_score_mean', 
                                   'changed_score_std', 'score_mean', 'score_std',
                                   'num_better', 'num_worse', 'num_unscored',
                                   'mean_better', 'std_better',
                                   'mean_worse', 'std_worse'])

for pp_method in pp_methods:
    ddscore_df.loc[pp_method] = score_pp_sb(pp_method, score_path, tol=50, crop=20)

print(score_base)
ddscore_df.sort_values(by='changed_score_mean', ascending=False)


Learnings:
- Previously assumed a low `frac_unscored` was good, but actually a substantial fraction of randomly generated image pairs don't need to be corrected much, so it makes sense that this does not go to zero
- As such, tolerances have been increased, and we basically reject less noise. Before we were just seeing noise
- ?stds so big because substantial fraction of image pairs get worse, and so are < 0.5 and spread the distribution right out
- separating out improved and worsened pairs we have substantiated the above.
- perhaps the most useful metric is the raw number of images that we improve the match of. i.e. you can always reject matches that haven't worked and not use them. in this case the final-superheavy pp is the best as we have the most improved images.
- doing this with images where we 100% should see changes (selecting large changes) will be interesting.

In [ ]:
# massage the DataFrame so we can copy it straight to LaTeX

dddfinal = ddscore_df.drop(columns=['score_mean', 'score_std', 'frac_unscored', 'mean_count']).sort_values(by='num_better', ascending=False)
dddfinal = dddfinal.sort_values(by='num_better', ascending=True).drop(columns=['changed_score_mean', 'changed_score_std'])
dddfinal[['num_better', 'num_worse', 'num_unscored']] = dddfinal[['num_better', 'num_worse', 'num_unscored']].astype(int)
dddfinal[['mean_better', 'std_better', 'mean_worse', 'std_worse']] = np.round(dddfinal[['mean_better', 'std_better', 'mean_worse', 'std_worse']],2)

dddfinal = dddfinal.iloc[[0,1,2,3,4]]

dddfinal.index = ['Unpreprocessed', 'Downsampled', 'Sequential Light', 'Sequential Heavy', 'Sequential Superheavy']
dddfinal = dddfinal.rename({'num_better': '$n_s$',
                            'num_worse': '$n_f$',
                            'num_unscored': '$n_u$',
                            'mean_better': 'success score',
                            'mean_worse': 'fail score'}, axis='columns')


dddfinal['success score'] = dddfinal['success score'].astype('str')
dddfinal['std_better'] = dddfinal['std_better'].astype('str')
dddfinal['fail score'] = dddfinal['fail score'].astype('str')
dddfinal['std_worse'] = dddfinal['std_worse'].astype('str')



dddfinal['success score'] = '$' + dddfinal['success score'] + ' \pm ' + dddfinal['std_better'] + '$'
dddfinal['fail score'] = '$' + dddfinal['fail score'] + ' \pm ' + dddfinal['std_worse'] + '$'

dddfinal = dddfinal.drop(columns=['std_better', 'std_worse'])
dddfinal = dddfinal[['$n_s$', 'success score', '$n_f$', 'fail score', '$n_u$']]
dddfinal

In [ ]:
s = dddfinal.to_latex('/Users/sam/Desktop/data.txt', bold_rows=True, column_format='lrrrrrr', escape=False)


We can also do things like look for a good match in any of the candidate pp strategies (i.e. we do not restrict ourselves to one pp strategy but search for a good match for image pair X across several strategies).

### Testing Unpp

In [ ]:
pp_methods = ['unpp']

score_base = 'final-superheavy'
score_path = os.path.join(base_path, 'data', score_base)

ddscore_df = pd.DataFrame(columns=['mean_count', 'frac_unscored', 'changed_score_mean', 
                                   'changed_score_std', 'score_mean', 'score_std',
                                   'num_better', 'num_worse', 'num_unscored',
                                   'mean_better', 'std_better',
                                   'mean_worse', 'std_worse'])

for pp_method in pp_methods:
    ddscore_df.loc[pp_method] = score_pp_sb(pp_method, score_path, tol=50, crop=20)

print(score_base)
ddscore_df.sort_values(by='changed_score_mean', ascending=False)


# NewHomographyFinder

Trying to get alignments without DeepMatchiing

In [ ]:
h = np.identity(3)
img_1_h = cv.warpPerspective(img_1, h, dsize=img_1.shape)

from sklearn import preprocessing

In [ ]:
def score_from_h_matrix(img_1, img_2, h):
    img_1_h = cv.warpPerspective(img_1, h, dsize=img_1.shape)
    score = dd_score(img_1[10:,:490], img_1_h[10:,:490], img_2[10:,:490])
    return score

def get_gradient(img_1, img_2, h, param_i, epsilon):
    h = h.reshape(-1,)
    
    h_up = h.copy()
    h_up[param_i] += epsilon
    
    h_down = h.copy()
    h_down[param_i] -= epsilon
    
    score_up = score_from_h_matrix(img_1, img_2, h_up.reshape(3,3))
    score_down = score_from_h_matrix(img_1, img_2, h_down.reshape(3,3))
    
    grad = (score_up - score_down) / (2 * epsilon)
    
    return grad

def get_all_gradients(img_1, img_2, h, epsilon):
    grads = [get_gradient(img_1, img_2, h, i, epsilon)
            for i in range(0,8)]
    
    return np.array(grads)
        
    
def update_h_matrix(img_1, img_2, h, alpha, epsilon):
    gradients = get_all_gradients(img_1, img_2, h, epsilon)
    #print('grads')
    #print(gradients)
    if np.all(gradients == 0):
        print('zero grad')
    deltas = gradients * alpha
    
    #min_max_scaler = preprocessing.StandardScaler((-1e1,1e2))
    #deltas = min_max_scaler.fit_transform(deltas[:, np.newaxis])

    deltas = np.append(deltas, [0])
    #print('deltas')
    #print(deltas)
    if np.any(np.abs(deltas) > 1e-3):
        print('deltas too large')
    if np.all(deltas == 0):
        print('deltas are 0')
    h_new = h.reshape(-1,) + deltas
    return h_new.reshape(3,3)

In [ ]:
def optimise_homography(img_1, img_2, alpha=1e-14, epsilon=1e-6, steps=50):
    
    h = np.identity(3)

    scores = []
    images = []

    for i in range(steps):
        #print('h')
        #print(h)
        h_new = update_h_matrix(img_1, img_2, h, alpha, epsilon)
        img_1_h = cv.warpPerspective(img_1, h_new, dsize=img_1.shape)
        score = dd_score(img_1[10:,:490], img_1_h[10:,:490], img_2[10:,:490])
        scores.append(score)
        images.append(img_1_h)
        h = h_new.copy()
        
    return scores, images

In [ ]:
steps = 100
scores, images = optimise_homography(img_1, img_2, epsilon=1e-4, alpha=1e-7, steps=steps)
plt.plot(range(steps), scores)

In [ ]:
best_index = np.array(scores).argmax()
best_image = images[best_index]

plt.figure(1, figsize=(17,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2))


plt.subplot(122)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(best_image, img_2))

In [ ]:
plt.figure(1, figsize=(17,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2))


plt.subplot(122)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(images[-1], img_2))

## New Pair

In [ ]:
steps = 2000
pair = img_pairs[12]

pp_path = 'clustered_norm'

img_1 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[0]) + '.png'), cv.IMREAD_GRAYSCALE)
img_2 = cv.imread(os.path.join(base_path, 'data', pp_path, str(pair[1]) + '.png'), cv.IMREAD_GRAYSCALE)

scores, images = optimise_homography(img_1, img_2, epsilon=1e-6, alpha=1e-14, steps=steps)

plt.plot(range(steps), scores)

In [ ]:
best_index = np.array(scores).argmax()
best_image = images[best_index]

plt.figure(1, figsize=(17,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2))


plt.subplot(122)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(best_image, img_2))

In [ ]:
plt.figure(1, figsize=(17,9))

plt.subplot(121)
plt.title(f'Overlaid Images - {pair[0]} and {pair[1]}')
plt.imshow(overlay(img_1, img_2))


plt.subplot(122)
plt.title(f'Transformed Overlay - {pair[0]} and {pair[1]}')
plt.imshow(overlay(images[0], img_2))

Gradients are a bit too crazy, will try a genetic algorithm

## GA

genetic algoritms to optimise the h matrix without using gradients (as they look unstable)